# Computational Social Science Project #2 

*Group number:* 8

*Group members:* Song, Jiyoung | Amerkhanian, Peter | Jiang, Qi

*Semester:* Fall 2022


Below we fill in some of the code you might use to answer some of the questions. Here are some additional resources for when you get stuck:
* Code and documentation provided in the course notebooks  
* [Markdown cheatsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) to help with formatting the Jupyter notebook
* Try Googling any errors you get and consult Stack Overflow, etc. Someone has probably had your question before!
* Send KQ a pull request on GitHub flagging the syntax that's tripping you up 

## 1. Introduction/Setup

#### a) Import relevant libraries
Add the other libraries you need for your code below and/or as you go. 

In [1]:
# import libraries you might need here 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# use random seed for consistent results 
np.random.seed(273)

#### b) Read in and inspect data frame 
Read in the data frame and look at some of its attributes. 

In [2]:
diabetes = pd.read_csv("Diabetes with Population Info by County 2017.csv", 
                       #CountyFips needs to be a string so leading 0 isn't dropped (this is only if you want to make choropleth map): 
                       dtype={"CountyFIPS": str}) 

In [3]:
# look at the dimensions of the diabetes data frame
print('shape: ', diabetes.shape) 

shape:  (3220, 95)


In [4]:
pd.set_option('display.max_rows', 100) # tells pandas how many rows to display when printing so results don't get truncated

# look at the data types for each column in diabetes df 
print('data types:', diabetes.dtypes)

data types: County                                                                                                                                                   object
State                                                                                                                                                    object
CountyFIPS                                                                                                                                               object
Diabetes_Number                                                                                                                                          object
Obesity_Number                                                                                                                                           object
Physical_Inactivity_Number                                                                                                                               object
sex and age_total population

Immediately, we see that some of the features that should be numeric (e.g., Diabetes_Number, Obesity_Number,  and Physical_Inactivity_Number) are not. We can check to see what the non-numeric values are in a column where we are expecting numeric information with a combination of `str.isnumeric()` and `unique()`.

In [5]:
# Return rows where the column "Diabetes_Number" is non-numeric and get the unique values of these rows
# the "~" below in front of diabetes negates the str.isnumeric() so it only takes non-numeric values
print(diabetes[~diabetes["Diabetes_Number"].str.isnumeric()]["Diabetes_Number"].unique()) 

['Suppressed']


In [6]:
# Now do the same as above, but for "Obesity_Number" :
print(diabetes[~diabetes['Obesity_Number'].str.isnumeric()]['Obesity_Number'].unique())

['No Data']


The values contained in the two columns above making them objects (rather than integers) appear to be strings like "No Data" and "Suppressed." Let's drop those rows in the next section, and also recode Physical_Inactivity_Number to be an integer. 

#### c. Recode variables

Convert 'Diabetes_Number', 'Obesity_Number', and 'Physical_Inactivity_Number' to integers below so we can use them in our analysis. Also fill in the object type we want to recode 'sex and age_total population_65 years and over_sex ratio (males per 100 females)' to. 

In [7]:
# Diabetes
# keep only useful info about our target feature, i.e., where diabetes_number not = 'Suppressed'
diabetes = diabetes[diabetes['Diabetes_Number']!="Suppressed"]  # note that the inside reference to the diabetes df identifies the column, and the outer calls specific rows according to a condition 

# use the astype method on Diabetes_Number to convert it to an integer...if you are not sure, what does the astype() documentation tell you are possible arguments? 
diabetes['Diabetes_Number'] = diabetes['Diabetes_Number'].astype(int) 

# Obesity
diabetes = diabetes[diabetes['Obesity_Number']!="No Data"]  # note that the inside reference to the diabetes df identifies the column, and the outer calls specific rows according to a condition 
diabetes['Obesity_Number'] = diabetes['Obesity_Number'].astype(int)

# Physical Inactivity
diabetes = diabetes[diabetes['Physical_Inactivity_Number']!="No Data"]  # note that the inside reference to the diabetes df identifies the column, and the outer calls specific rows according to a condition 
diabetes['Physical_Inactivity_Number'] = diabetes['Physical_Inactivity_Number'].astype(int)


# 65+ sex ratio had one "-" in it so let's drop that row first
diabetes = diabetes[diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)']!= "-"]
# change to numeric (specifically, integer or float?) from string (because originally included the "-" )
diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)'] = diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)'].astype(float)

We should probably scale our count variables to be proportional to county population. We create the list 'rc_cols' to select all the features we want to rescale, and then use the `.div()` method to avoid typing out every single column we want to recode. 

In [8]:
# select count variables to rc to percentages; make sure we leave out ratios and our population variable b/c these don't make sense to scale by population
rc_cols = [col for col in diabetes.columns if col not in ['County', 'State', 'CountyFIPS', 
                                                        'sex and age_total population_65 years and over_sex ratio (males per 100 females)', 'sex and age_total population_sex ratio (males per 100 females)', 'sex and age_total population_18 years and over_sex ratio (males per 100 females)',  
                                                        'race_total population']]
           
diabetes[rc_cols] = diabetes[rc_cols].apply(pd.to_numeric, errors='coerce') # recode all selected columns to numeric

# divide all columns but those listed above by total population to calculate rates
diabetes[rc_cols] = diabetes[rc_cols].div(diabetes['race_total population'], axis=0)

Let's check our work. Are all rates bounded by 0 and 1 as expected? 

In [9]:
pd.set_option('display.max_columns', None)
# inspect recoded values
diabetes_summary = diabetes.describe().transpose() # note we use the transpose method rather than .T because this object is not a numpy array
  
# check recoding 
with pd.option_context('display.max_rows', 100, 'display.max_columns', None): 
    display(diabetes_summary.iloc[ : ,[0,1,3,7]]) # select which columns in the summary table we want to present

count           mean  \
Diabetes_Number                                     3112.0       0.092857   
Obesity_Number                                      3112.0       0.251666   
Physical_Inactivity_Number                          3112.0       0.200861   
sex and age_total population_65 years and over_...  3112.0      84.275643   
race_total population                               3112.0  103682.624036   
race_total population_one race                      3112.0       0.976059   
race_total population_two or more races             3112.0       0.023941   
race_total population_one race_1                    3112.0       0.976059   
race_total population_one race_white                3112.0       0.831107   
race_total population_one race_black or african...  3112.0       0.090215   
race_total population_one race_american indian ...  3112.0       0.019398   
race_total population_one race_american indian ...  3112.0       0.001791   
race_total population_one race_american indian ...  3112.0       0.001338   
race_total population_one race_american indian ...  3112.0       0.001229   
race_total population_one race_american indian ...  3112.0       0.003315   
race_total population_one race_asian                3112.0       0.013749   
race_total population_one race_asian_asian indian   3112.0       0.002628   
race_total population_one race_asian_chinese        3112.0       0.002644   
race_total population_one race_asian_filipino       3112.0       0.002744   
race_total population_one race_asian_japanese       3112.0       0.000707   
race_total population_one race_asian_korean         3112.0       0.001174   
race_total population_one race_asian_vietnamese     3112.0       0.001255   
race_total population_one race_asian_other asian    3112.0       0.002598   
race_total population_one race_native hawaiian ...  3112.0       0.000874   
race_total population_one race_native hawaiian ...  3112.0       0.000303   
race_total population_one race_native hawaiian ...  3112.0       0.000142   
race_total population_one race_native hawaiian ...  3112.0       0.000134   
race_total population_one race_native hawaiian ...  3112.0       0.000295   
race_total population_one race_some other race      3112.0       0.020716   
race_total population_two or more races_1           3112.0       0.023941   
race_total population_two or more races_white a...  3112.0       0.006444   
race_total population_two or more races_white a...  3112.0       0.008618   
race_total population_two or more races_white a...  3112.0       0.003021   
race_total population_two or more races_black o...  3112.0       0.000652   
race alone or in combination with one or more o...  3112.0       1.000000   
race alone or in combination with one or more o...  3112.0       0.852907   
race alone or in combination with one or more o...  3112.0       0.098875   
race alone or in combination with one or more o...  3112.0       0.029923   
race alone or in combination with one or more o...  3112.0       0.018213   
race alone or in combination with one or more o...  3112.0       0.002067   
race alone or in combination with one or more o...  3112.0       0.023626   
hispanic or latino and race_total population        3112.0       1.000000   
hispanic or latino and race_total population_hi...  3112.0       0.093201   
hispanic or latino and race_total population_hi...  3112.0       0.069331   
hispanic or latino and race_total population_hi...  3112.0       0.005656   
hispanic or latino and race_total population_hi...  3112.0       0.001568   
hispanic or latino and race_total population_hi...  3112.0       0.016645   
hispanic or latino and race_total population_no...  3112.0       0.906799   
hispanic or latino and race_total population_no...  3112.0       0.765065   
hispanic or latino and race_total population_no...  3112.0       0.088817   
hispanic or latino and race_total population_no...  3112.0       0.018016   
hispanic or latino and race_total population_no..

#### d. Check for duplicate columns

There are a lot of columns in this data frame. Let's see if there are any are duplicates. 

In [11]:
# I used Google to figure this out, and adapted this example for our purposes:  
# source: https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/ 
def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
    return list(duplicateColumnNames)

duplicateColumnNames = list(getDuplicateColumns(diabetes))
print('Duplicate Columns are as follows: ')
duplicateColumnNames

Duplicate Columns are as follows: 


['race_total population_one race_1',
 'sex and age_total population',
 'race_total population_two or more races_1',
 'hispanic or latino and race_total population',
 'sex and age_total population_65 years and over_1',
 'sex and age_total population_18 years and over_1']

In [12]:
# now drop list of duplicate features from our df using the .drop() method
diabetes = diabetes.drop(columns=duplicateColumnNames) 

## 2. Exploratory Data Analysis

In [16]:
diabetes.corr()

Diabetes_Number  \
Diabetes_Number                                            1.000000   
Obesity_Number                                             0.549577   
Physical_Inactivity_Number                                 0.591118   
sex and age_total population_65 years and over_...        -0.113471   
race_total population                                     -0.154708   
race_total population_one race                             0.107401   
race_total population_two or more races                   -0.107401   
race_total population_one race_white                      -0.230673   
race_total population_one race_black or african...         0.369464   
race_total population_one race_american indian ...         0.008445   
race_total population_one race_american indian ...         0.062850   
race_total population_one race_american indian ...        -0.007163   
race_total population_one race_american indian ...         0.012984   
race_total population_one race_american indian ...         0.028245   
race_total population_one race_asian                      -0.238331   
race_total population_one race_asian_asian indian         -0.186293   
race_total population_one race_asian_chinese              -0.199405   
race_total population_one race_asian_filipino             -0.098891   
race_total population_one race_asian_japanese             -0.075401   
race_total population_one race_asian_korean               -0.200529   
race_total population_one race_asian_vietnamese           -0.161593   
race_total population_one race_asian_other asian          -0.205197   
race_total population_one race_native hawaiian ...        -0.063392   
race_total population_one race_native hawaiian ...        -0.030220   
race_total population_one race_native hawaiian ...        -0.055445   
race_total population_one race_native hawaiian ...        -0.069046   
race_total population_one race_native hawaiian ...        -0.072684   
race_total population_one race_some other race            -0.162498   
race_total population_two or more races_white a...         0.023969   
race_total population_two or more races_white a...        -0.005600   
race_total population_two or more races_white a...        -0.287866   
race_total population_two or more races_black o...         0.060250   
race alone or in combination with one or more o...              NaN   
race alone or in combination with one or more o...        -0.244925   
race alone or in combination with one or more o...         0.365546   
race alone or in combination with one or more o...         0.006052   
race alone or in combination with one or more o...        -0.243781   
race alone or in combination with one or more o...        -0.060953   
race alone or in combination with one or more o...        -0.169926   
hispanic or latino and race_total population_hi...        -0.221210   
hispanic or latino and race_total population_hi...        -0.192196   
hispanic or latino and race_total population_hi...        -0.074168   
hispanic or latino and race_total population_hi...        -0.043513   
hispanic or latino and race_total population_hi...        -0.148911   
hispanic or latino and race_total population_no...         0.221210   
hispanic or latino and race_total population_no...        -0.078337   
hispanic or latino and race_total population_no...         0.371810   
hispanic or latino and race_total population_no...         0.011195   
hispanic or latino and race_total population_no...        -0.238610   
hispanic or latino and race_total population_no...        -0.061978   
hispanic or latino and race_total population_no...        -0.046533   
hispanic or latino and race_total population_no...        -0.056689   
hispanic or latino and race_total population_no...        -0.074903   
hispanic or latino and race_total population_no...        -0.053113   
total housing units                                        0.090290   
citizen -  voting age population_citizen -  18 ...         0.273325   
citiz

## 3. Prepare to Fit Models

### 3.1 Finalize Data Set

We've already cleaned up the data, but we can make a few more adjustments before partitioning the data and training models. Let's recode 'State' to be a categorical variable using `pd.get_dummies` and drop 'County' using `.drop()` because 'CountyFIPS' is already a unique identifier for the county. 

In [181]:
# create dummy features out of 'State' , which might be related to diabetes rates 
diabetes_clean = pd.get_dummies(diabetes, 
                               columns = ['State'],  
                               drop_first = True) # only create 49 dummies by dropping first in category

# drop 'County' variable
diabetes_clean = diabetes_clean.drop(labels = ['County', 'CountyFIPS'],
                               axis = 1) # which axis tells python we want to drop columns rather than index rows?

# look at first 10 rows of new data frame 
diabetes_clean.head(10)


Diabetes_Number  Obesity_Number  Physical_Inactivity_Number  \
0         0.093804        0.244366                    0.226902   
1         0.080281        0.233255                    0.192324   
2         0.131875        0.310643                    0.210651   
3         0.105251        0.290052                    0.258667   
4         0.108977        0.246804                    0.249198   
5         0.224208        0.353748                    0.321967   
6         0.145318        0.349513                    0.300974   
7         0.127648        0.298580                    0.234140   
8         0.141518        0.324425                    0.220570   
9         0.122848        0.300429                    0.237342   

   sex and age_total population_65 years and over_sex ratio (males per 100 females)  \
0                                               76.5                                  
1                                               86.6                                  
2                                               76.2                                  
3                                               80.5                                  
4                                               79.7                                  
5                                               73.6                                  
6                                               69.1                                  
7                                               73.7                                  
8                                               73.3                                  
9                                               86.7                                  

   race_total population  race_total population_one race  \
0                  55200                        0.981341   
1                 208107                        0.982836   
2                  25782                        0.986308   
3                  22527                        0.991699   
4                  57645                        0.983780   
5                  10352                        1.000000   
6                  20025                        0.983870   
7                 115098                        0.976229   
8                  33826                        0.989742   
9                  25853                        0.991955   

   race_total population_two or more races  \
0                                 0.018659   
1                                 0.017164   
2                                 0.013692   
3                                 0.008301   
4                                 0.016220   
5                                 0.000000   
6                                 0.016130   
7                                 0.023771   
8                                 0.010258   
9                                 0.008045   

   race_total population_one race_white  \
0                              0.768786   
1                              0.862662   
2                              0.473819   
3                              0.766547   
4                              0.955052   
5                              0.219861   
6                              0.520050   
7                              0.743349   
8                              0.575977   
9                              0.928209   

   race_total population_one race_black or african american  \
0                                           0.191395          
1                                           0.094970          
2                                           0.475758          
3                                           0.222755          
4                                           0.014954          
5                                           0.762461          
6                                           0.452185          
7                                           0.205703          
8                                           0.393750          
9                                           0.

In [ ]:
drop_col = diabetes_clean.T[diabetes_clean.var() <= 0].index[0]
features = diabetes_clean.drop(['Diabetes_Number', drop_col], axis=1)

### 3.2/3.3 Partition Data and Feature Selection

Now, we will partition our data to prepare it for the training process. We will use 60% train—20% validation—20% test in this case. More data in the training set lowers bias, but then increases variance in the validation/test sets. Balancing between bias and variance with choice of these set sizes is important as we want to ensure that there is enough data to train on to get good predictions, but also want to make sure our hold-out sets are representative enough.

In [73]:
from sklearn.model_selection import train_test_split

# Set y 
y = diabetes_clean['Diabetes_Number']

# X (everything except diabetes, our target)
X = diabetes_clean.drop(['Diabetes_Number'], axis=1)

We should also preprocess our data. Using the `preprocessing` module from sklearn, let's scale our features so that they are mean-centered.

In [75]:
from sklearn import preprocessing

X = preprocessing.scale(X)
X.shape

(3112, 135)

We can also get rid of the 0 variance features using the `VarianceThreshold()` method from `feature_selection`. 

In [76]:
X

array([[-0.16471873,  0.58515483, -0.75155595, ..., -0.13288567,
        -0.15389675, -0.08242523],
       [-0.4154411 , -0.19181486,  0.22466112, ..., -0.13288567,
        -0.15389675, -0.08242523],
       [ 1.33084974,  0.21998678, -0.7805525 , ..., -0.13288567,
        -0.15389675, -0.08242523],
       ...,
       [-0.28640652, -0.67657217,  1.28786783, ..., -0.13288567,
        -0.15389675, 12.13220822],
       [-0.85439566,  0.20787775,  0.45663349, ..., -0.13288567,
        -0.15389675, 12.13220822],
       [ 0.72518908, -0.1364354 ,  0.61128174, ..., -0.13288567,
        -0.15389675, 12.13220822]])

In [77]:
from sklearn import feature_selection

selector = feature_selection.VarianceThreshold(0)
X = selector.fit_transform(X)

In [78]:
X.shape

(3112, 134)

And finally, let's split our data:

In [93]:
# split the data
# train_test_split returns 4 values: X_train, X_test, y_train, y_test, so how do we create a 60-20-20 train-validate-test split? 
np.random.seed(10)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.6, test_size=.4)

X_test, X_validate, y_test, y_validate = train_test_split(X_test, y_test, train_size=.5, test_size=.5)

## Checking the sizes:

In [109]:
print("Train", X_train.shape[0] / X.shape[0])
X_train.shape[0]

Train 0.5999357326478149


1867

In [110]:
print("Val", X_validate.shape[0] / X.shape[0])
X_validate.shape[0]

Val 0.20019280205655526


623

In [111]:
print("Test", X_test.shape[0] / X.shape[0])
X_test.shape[0]

Test 0.19987146529562982


622

## 4. Train Models

In [114]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [154]:
# ElasticNet
elasticnet_reg = ElasticNet(l1_ratio=.5, max_iter=100000)
elasticnet_model = elasticnet_reg.fit(X_train, y_train)
# Ridge
ridge_reg = Ridge()
ridge_model = ridge_reg.fit(X_train, y_train)
# LASSO
lasso_reg = Lasso(max_iter=10000)
lasso_model = lasso_reg.fit(X_train, y_train)

In [155]:
def rmse(pred, actual):
    return np.sqrt(np.mean((pred - actual)**2))

In [156]:
for model in zip([elasticnet_model, ridge_model, lasso_model],
                 ['Elastic Net', 'Ridge', 'Lasso']):
    train_rmse = rmse(model[0].predict(X_train), y_train)
    print(model[1], "Training RMSE:", train_rmse)

Elastic Net Training RMSE: 0.028265809392299165
Ridge Training RMSE: 0.018675957736169712
Lasso Training RMSE: 0.028265809392299165


## 5. Validate and Refine Models

In [184]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


def rmse(pred, actual):
    return np.sqrt(np.mean((pred - actual)**2))


# Elastic Net
elasticnet_reg_data = pd.DataFrame([elasticnet_model.coef_, features.columns]).T
elasticnet_reg_data.columns = ['Coefficient', 'Feature']
elasticnet_pred = elasticnet_model.predict(X_validate)
print('Elastic NEt RMSE:', rmse(elasticnet_pred, y_validate))
elasticnet_reg.get_params()
param_grid = {'fit_intercept': ['True', 'False']}
elasticnet_grid_reg = GridSearchCV(elasticnet_reg, param_grid, cv=3)
elasticnet_grid_reg.fit(X_train, y_train)
best_index = np.argmax(elasticnet_grid_reg.cv_results_["mean_test_score"])
best_elasticnet_pred = elasticnet_grid_reg.best_estimator_.predict(X_validate)
print(elasticnet_grid_reg.cv_results_["params"][best_index])
print('Best CV R^2:', max(elasticnet_grid_reg.cv_results_["mean_test_score"]))
print('Validation R^2:', elasticnet_grid_reg.score(X_validate, y_validate))
print('Validation RMSE', rmse(best_elasticnet_pred, y_validate))
# Ridge
ridge_reg_data = pd.DataFrame([ridge_model.coef_, features.columns]).T
ridge_reg_data.columns = ['Coefficient', 'Feature']
ridge_pred = ridge_model.predict(X_validate)
print("************")
print('Ridge RMSE:', rmse(ridge_pred, y_validate))
ridge_reg.get_params()
param_grid = {
    'alpha': np.arange(.1, 1, .1),
    'fit_intercept': ['True', 'False'],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr']
}
ridge_grid_reg = GridSearchCV(ridge_reg, param_grid, cv=3)
ridge_grid_reg.fit(X_train, y_train)
best_index = np.argmax(ridge_grid_reg.cv_results_["mean_test_score"])
best_ridge_pred = ridge_grid_reg.best_estimator_.predict(X_validate)
print('Best CV R^2:', max(ridge_grid_reg.cv_results_["mean_test_score"]))
print('Validation R^2:', ridge_grid_reg.score(X_validate, y_validate))
print('Validation RMSE', rmse(best_ridge_pred, y_validate))
ridge_grid_reg.cv_results_["mean_test_score"]
# LASSO
lasso_reg_data = pd.DataFrame([lasso_model.coef_, features.columns]).T
lasso_reg_data.columns = ['Coefficient', 'Feature']
lasso_pred = lasso_model.predict(X_validate)
print("************")
print('LASSO RMSE:', rmse(lasso_pred, y_validate))
lasso_reg.get_params()
param_grid = {
    'alpha': np.arange(.1, 1, .1),
    'fit_intercept': ['True', 'False']
}
lasso_grid_reg = GridSearchCV(lasso_reg, param_grid, cv=3)
lasso_grid_reg.fit(X_train, y_train)
best_index = np.argmax(lasso_grid_reg.cv_results_["mean_test_score"])
best_lasso_pred = lasso_grid_reg.best_estimator_.predict(X_validate)
print('Best CV R^2:', max(lasso_grid_reg.cv_results_["mean_test_score"]))
print('Validation R^2:', lasso_grid_reg.score(X_validate, y_validate))
print('Validation RMSE', rmse(best_lasso_pred, y_validate))
lasso_grid_reg.cv_results_["mean_test_score"]

Elastic NEt RMSE: 0.029120756799777724
{'fit_intercept': 'True'}
Best CV R^2: -0.0017396620421605096
Validation R^2: -0.00023919730522692362
Validation RMSE 0.029120756799777724
************
Ridge RMSE: 0.020866121382215955
Best CV R^2: 0.46689681114478104
Validation R^2: 0.48909060263632576
Validation RMSE 0.02081242414621408
************
LASSO RMSE: 0.029120756799777724
Best CV R^2: -0.0017396620421605096
Validation R^2: -0.00023919730522692362
Validation RMSE 0.029120756799777724


array([-0.00173966, -0.00173966, -0.00173966, -0.00173966, -0.00173966,
       -0.00173966, -0.00173966, -0.00173966, -0.00173966, -0.00173966,
       -0.00173966, -0.00173966, -0.00173966, -0.00173966, -0.00173966,
       -0.00173966, -0.00173966, -0.00173966])

In [187]:
np.argmax(ridge_grid_reg.cv_results_["mean_test_score"])

67

In [194]:
pd.DataFrame(ridge_grid_reg.cv_results_).sort_values(by='rank_test_score', ascending=True)

mean_fit_time  std_fit_time  mean_score_time  std_score_time param_alpha  \
71       0.010412  7.362156e-03         0.000000        0.000000         0.9   
67       0.010414  7.363785e-03         0.000000        0.000000         0.9   
69       0.040448  4.792466e-03         0.000000        0.000000         0.9   
65       0.041657  7.364909e-03         0.000000        0.000000         0.9   
64       0.000000  0.000000e+00         0.005207        0.007364         0.9   
70       0.005210  7.368056e-03         0.000000        0.000000         0.9   
68       0.005372  7.596885e-03         0.000000        0.000000         0.9   
66       0.005207  7.364459e-03         0.000000        0.000000         0.9   
63       0.008921  6.567815e-03         0.005208        0.007366         0.8   
59       0.010414  7.363785e-03         0.000000        0.000000         0.8   
57       0.041657  7.364178e-03         0.000000        0.000000         0.8   
61       0.041656  7.364459e-03         0.000000        0.000000         0.8   
58       0.005207  7.363673e-03         0.000000        0.000000         0.8   
56       0.005207  7.363673e-03         0.000000        0.000000         0.8   
62       0.005207  7.363560e-03         0.000000        0.000000         0.8   
60       0.005208  7.365134e-03         0.000000        0.000000         0.8   
55       0.010415  7.364182e-03         0.000000        0.000000         0.7   
51       0.006214  6.767507e-03         0.000000        0.000000         0.7   
53       0.041657  7.363054e-03         0.000000        0.000000         0.7   
49       0.044873  6.024865e-03         0.000359        0.000508         0.7   
52       0.005208  7.364572e-03         0.000000        0.000000         0.7   
50       0.005363  7.585084e-03         0.000148        0.000209         0.7   
48       0.000000  0.000000e+00         0.005208        0.007365         0.7   
54       0.005207  7.364235e-03         0.005207        0.007363         0.7   
47       0.006441  9.109564e-03         0.000341        0.000483         0.6   
43       0.007024  6.475592e-03         0.000000        0.000000         0.6   
39       0.010412  7.362717e-03         0.000000        0.000000         0.5   
35       0.010413  7.363111e-03         0.000000        0.000000         0.5   
45       0.036450  7.364010e-03         0.005207        0.007363         0.6   
41       0.041657  7.363111e-03         0.000000        0.000000         0.6   
46       0.002490  1.981633e-03         0.005536        0.007308         0.6   
40       0.005974  8.448589e-03         0.000000        0.000000         0.6   
42       0.005679  8.031728e-03         0.000189        0.000268         0.6   
44       0.005207  7.363560e-03         0.000000        0.000000         0.6   
31       0.005207  7.363448e-03         0.000000        0.000000         0.4   
27       0.005207  7.363897e-03         0.000000        0.000000         0.4   
33       0.036450  7.363335e-03         0.005207        0.007364         0.5   
37       0.046864  5.947204e-07         0.000000        0.000000         0.5   
32       0.005207  7.364122e-03         0.000000        0.000000         0.5   
34       0.005208  7.365808e-03         0.000000        0.000000         0.5   
36       0.005207  7.364010e-03         0.000000        0.000000         0.5   
38       0.005210  7.367382e-03         0.000000        0.000000         0.5   
19       0.012031  2.117340e-03         0.000332        0.000470         0.3   
23       0.016759  3.654322e-03         0.000666        0.000471         0.3   
25       0.042353  6.378560e-03         0.000000        0.000000         0.4   
29       0.041652  7.361089e-03         0.000000        0.000000         0.4   
26       0.005207  7.363897e-03         0.000000        0.000000         0.4   
30       0.005208  7.365134e-03         0.000000        0.000000         0.4   
24       0.004502  3.796305e-04         0.000450        0.000352         0.4   
28  

In [ ]:
# use X_validation and y_validation data sets to evaluate and refine your models

## 6. Discussion Questions

In [1]:
# insert responses for discussion Qs here